In [31]:
### string.fromat

import yaml
import import_ipynb
import Read_pdf as rp

def display_template(data):
    topic = data.keys()
    print('--------------模板列表-----------------')
    for item in topic:
        print(item + ':')
        for sub_item in data[item]:
            print(' -' + sub_item)

def get_variable(text):#抓取{ 及 }的位置並取出裏頭的字串變成dic的key
    my_pos = []
    my_pos2 = []
    pos = 0
    pos2 = 0
    while True:
        pos = text.find('{', pos)
        pos2 = text.find('}', pos2)
        if pos2 == -1:
            break
        my_pos.append(pos)
        my_pos2.append(pos2)
        pos += 1
        pos2 +=1

    keys = []
    for start,end in zip(my_pos, my_pos2):
        keys.append(text[start+1:end])
    d = dict.fromkeys(keys)
    del my_pos
    del my_pos2
    del keys
    return d


def bid_value(bid, dic):
    if bid.currency == '人民幣':
        prefixCur = 'RMB'
        suffixCur = ''
    else:
        prefixCur =''
        suffixCur = '元'

    def name_format(var):
        return '「' + str(var) +'」'

    def price_format(var):
        return prefixCur + str(var) + suffixCur

    for key in dic:
        if key == '次別':
            dic[key] = bid.sn
        elif key == '決包廠商':
            dic[key] = name_format(bid.award_vnd)
        elif key == '追加減':
            dic[key] = bid.change
        elif key == '公司別':
            dic[key] = bid.comp
        elif key == '廠區':
            dic[key] = bid.id[1:3]
        elif key == '預算金額':
            dic[key] = price_format(bid.budget)
        elif key == '報價金額':
            dic[key] = price_format(bid.vnd[bid.cheapest_sn]['quo'])
        elif key == '議價金額':
            dic[key] = price_format(bid.vnd[bid.cheapest_sn]['finalPrice'])
        elif key == '差異':
            dic[key] = bid.dif_text(bid.cheapest_sn)
        elif key == '工程名稱':
            dic[key] = bid.name
        elif key == '最低廠商':
            dic[key] = name_format(bid.vnd[bid.cheapest_sn]['name'])
        elif key == '報價家數':
                dic[key] = len(bid.vnd)
        elif key == '及催報':
            dic[key] = ''
            for vnd in bid.vnd:
                if vnd['type'] == '後報' or vnd['type'] == '後加':
                    dic[key] = '及催報'
                    break
        
        elif key == '議價狀況':
            sum_bargain_times = 0 #議價總次數
            for vnd in bid.vnd:
                sum_bargain_times += int(vnd['bargain_times'])
                
            if bid.cheapest_sn == 0 and int(bid.vnd[bid.cheapest_sn]['bargain_times']) == sum_bargain_times:
                dic[key] = '經洽議報價最低廠商'
            else:
                dic[key] = '議後以廠商'
        
        elif key == '最低':
            if dic['議價狀況'] == '議後以廠商':
                dic[key] = '最低'
            else:
                dic[key] = ''
        
        elif key == '小規模納稅人':
            if int(bid.vnd[bid.cheapest_sn]['finalRate']) == 3:
                dic[key] = '小規模納稅人'
            else:
                dic[key] = ''
        
        elif key == '廠商稅率':
            if bid.vnd[bid.cheapest_sn]['finalPrice'] != None:
                dic[key] = bid.vnd[bid.cheapest_sn]['finalRate']
            else:
                dic[key] = bid.vnd[bid.cheapest_sn]['quoRate']
        
        elif key =='發票符合/洽會同意':
            setting_rate = dic['預算稅率']
#             print('最低廠商議後稅率:', bid.vnd[bid.cheapest_sn]['finalRate'])
#             print('預算稅率:', setting_rate)
            if setting_rate == bid.vnd[bid.cheapest_sn]['finalRate']:
                dic[key] = '符合'
            else:
                dic[key] = '(洽會工程部門確認同意)'
        
        elif key == '回議':
            if bid.cheapest_sn == 0:
                dic[key] = ''
            else:
                if bid.vnd[0]['finalPrice'] == None:
                    dic[key] = '，回議廠商{name}未回覆，不再回議'.format(name = name_format(bid.vnd[0]['name']))
                else:
                    dic[key] = '，回議廠商{name}回覆{price}價高'.format(name = name_format(bid.vnd[0]['name']),\
                                                              price = price_format(bid.vnd[0]['finalPrice']))
        else:
            dic[key] = input('請輸入「{}」的值:'.format(key))
    return dic

if __name__ == '__main__':
    file_yaml = 'template.yaml'
    with open(file_yaml, 'r', encoding = 'utf8') as stream:
        data = yaml.load(stream, Loader = yaml.FullLoader)
    
#     print('--------------模板可用樣式列表-------------')
#     display_template(data)        

    pdf_file = 'd:/T3.pdf'
    bidcase = rp.open(pdf_file)
    
    is_bargain = lambda n :'有議價' if n else '無議價'
    is_overbudget = lambda n :'(超預算)' if n else '(低預算)'
    a = ''
    if bidcase.type == '一般案':
        a = is_overbudget(bidcase.is_overBudget(bidcase.cheapest_sn))
    
    print('--------------模板樣式[{type}][{bargain}]-------------'\
        .format(type = bidcase.type, bargain = is_bargain(bidcase.is_bargain) + a))
            
    text = data[bidcase.type][is_bargain(bidcase.is_bargain) + a]
    
    print(text)
    
    dict_tags = get_variable(text)
    dict_tags = bid_value(bidcase, dict_tags)

    print('----------------套版後-------------------')
    print(text.format(**dict_tags))

--------------模板樣式[變更案][有議價]-------------
1.本案為{次別}次變更案，預算編列{追加減}{預算金額}。
2.經洽議原承商{決包廠商}後由{報價金額}降至{議價金額}，{差異}。
3.擬以{議價金額}向{決包廠商}辦理{追加減}(同00案發票稅率{廠商稅率}%)。

----------------套版後-------------------
1.本案為02次變更案，預算編列追減RMB-1,010。
2.經洽議原承商「安徽防腐公司」後由RMB-1,010降至RMB-1,010，同預算價格。
3.擬以RMB-1,010向「安徽防腐公司」辦理追減(同00案發票稅率11%)。

